# 要約 
このJupyter Notebookは、LMSYSの「Chatbot Arena」コンペティションにおける、2つのチャットボットの応答の優劣を予測する問題に取り組んでいます。具体的には、与えられたプロンプトやチャットボットの応答からキーワードを抽出し、これらの情報を基に予測モデルを構築しています。

主な手法としては、以下の要素が使用されています：

1. **KeyBERTライブラリ**: キーワード抽出のために、KeyBERTを使用しています。このモデルは、文書から重要なキーワードを抽出するためのもので、事前トレーニングされたDistilBERTモデルと組み合わせて利用されています。

2. **PyTorchおよびTransformersライブラリ**: PyTorchを基盤とした深層学習モデルを使用して、提案されたデータセットに対して応答の優劣を予測します。具体的には、RoBERTaモデルを用いてシーケンス分類を行っています。

3. **データの処理**: テストデータを整形し、プロンプトや応答からキーワードを抽出した後、これらのキーワードを新しい特徴量としてデータフレームに追加しています。抽出したキーワードは、モデルの入力として使用されます。

4. **データセットクラスの定義**: BERTに基づくデータセットクラスを定義し、トークン化やパディングなど前処理を行っています。これにより、モデルに与えるためにデータが適切に整形されています。

5. **予測関数の実装**: 除外ファイルからモデルの保存状態を読み込み、テストデータに対して予測を行います。最終的に、モデルAとモデルBの勝者の予測結果を集約し、適切な形式で出力しています。

このNotebookでは、指定されたタスクに対する実行時間の制限を考慮しつつ、ライブラリのインストールやデータ処理を行い、提出用のCSVファイルの生成までを行っています。最終的に、モデルによる予測に基づいた結果を含んだ提出ファイルを作成しています。

---


# 用語概説 
以下は、Jupyter Notebook内で使用されている機械学習・深層学習の専門用語に関する簡単な解説です。初心者がつまずきやすい、ややマイナーまたは実務経験がなければ理解しにくい内容に焦点を当てています。

### 専門用語解説

1. **KeyBERT**
   - 文書から重要なキーワードを抽出するためのライブラリ。BERTモデルを用いて文のコンテキストを理解し、最も関連性の高い単語やフレーズを特定します。

2. **Gradient Scaler (GradScaler)**
   - 自動混合精度トレーニングを行うためのPyTorchの機能。計算の精度を保ちながらメモリ使用量を減らし、学習の速度を向上させるために、モデルの計算を浮動小数点16ビットに変換する際のスケールを管理します。

3. **Deterministic**
   - モデルやアルゴリズムが常に同じ結果を返す特性。特に乱数を使用する処理で再現性を確保するために、特定の乱数シードを設定して実行します。

4. **Tokenizer**
   - テキストをトークン（単語やサブワードなどの単位）に変換するためのコンポーネント。BERTやRoBERTaのようなモデルの入力としてテキストを適切な形式に変換する役割を持っています。

5. **Attention Mask**
   - トークン化された文において、モデルがどのトークンに注意を払うべきかを示すマスク。実際のデータのトークンとパディングトークンを区別するために使用され、モデルの計算効率化に寄与します。

6. **Logits**
   - モデルの出力で、各クラスの信頼度を表現した生のスコア。通常、ロジスティック回帰や分類タスクにおいて、これらのスコアは確率に変換されます。

7. **BERTDataSet**
   - PyTorchのDatasetクラスを拡張したカスタムクラス。文とそのターゲットラベルを管理し、BERTモデル用にエンコードされた入力を生成するための方法を提供します。

8. **DataLoader**
   - PyTorchの機能で、大規模なデータセットを効率的にバッチ処理し、モデルに読み込むためのツール。シャッフルやマルチプロセッシングを使用してデータの供給を行います。

9. **Mean Squared Error (MSE)**
   - 回帰分析やモデル評価において使用される指標。予測値と実際の値との差の二乗平均をとることで、モデルの誤差を測定します。

10. **Stratified K-Fold**
    - クロスバリデーションの手法の一つで、データセットを一様に分割し、分布を保ったままモデルの性能を評価します。特にクラスの不均衡に対処する際に使用されます。

これらの用語は、初心者には馴染みのない場合が多く、特に実務経験がないと理解が難しいことがあります。この解説が、Jupyter Notebookを読む手助けとなることを願っています。

---


# LMSYS キーワード torch RoBERTa での提出
インターネットオフ条件で

- https://www.kaggle.com/code/stpeteishii/lmsys-prompt-response-words-keybert <br/>
トレーニングデータの処理

- https://www.kaggle.com/code/stpeteishii/lmsys-keywords-torch-roberta <br/>
処理されたトレーニングデータを使用したモデルのトレーニング

- https://www.kaggle.com/code/stpeteishii/download-keybert <br/>
KeyBERTのダウンロード

- https://www.kaggle.com/code/stpeteishii/save-distilbert-base-nli-mean-tokens <br/>
distilbert-base-nli-mean-tokensのダウンロード

- https://www.kaggle.com/code/stpeteishii/lmsys-keywords-torch-roberta-for-submission <br/>
テストデータの処理、推論（このノートブック）

In [ ]:
!pip install keybert --no-index --find-links=file:///kaggle/input/download-keybert
# KeyBERTライブラリをインストールします。
# --no-indexフラグは、PyPIインデックスからの検索を無効にします。
# --find-linksオプションは、指定されたローカルのファイルパスからパッケージをインストールするために使用します。

In [ ]:
from keybert import KeyBERT
# KeyBERTライブラリからKeyBERTクラスをインポートします。
# KeyBERTは、文書からキーワードを抽出するためのモデルです。

In [ ]:
import numpy as np 
import pandas as pd 
import os
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import matplotlib.pyplot as plt 
import transformers
import random
import warnings

warnings.simplefilter('ignore')  # 警告を無視するように設定します。

scaler = torch.cuda.amp.GradScaler()  # 自動混合精度トレーニングのスケーラーを作成します。
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # GPUが利用可能であればGPUを使用し、そうでなければCPUを使用します。
device  # 現在のデバイス（GPUまたはCPU）を表示します。

In [ ]:
def random_seed(SEED):
    # 乱数シードを設定する関数です。
    random.seed(SEED)  # Pythonのrandomモジュールのシードを設定します。
    os.environ['PYTHONHASHSEED'] = str(SEED)  # 環境変数でハッシュシードを設定します。
    np.random.seed(SEED)  # NumPyの乱数シードを設定します。
    torch.manual_seed(SEED)  # PyTorchのCPU乱数シードを設定します。
    torch.cuda.manual_seed(SEED)  # PyTorchのGPU乱数シードを設定します。
    torch.cuda.manual_seed_all(SEED)  # 全てのGPUの乱数シードを設定します。
    torch.backends.cudnn.deterministic = True  # CuDNNの決定論的動作を有効にします。
    
SEED = 508  # 使用する乱数シードを508に設定します。
random_seed(SEED)  # 上記のシードで乱数の再現性を確保します。

# テストデータの処理

In [ ]:
from sentence_transformers import SentenceTransformer

# SentenceTransformerライブラリからSentenceTransformerクラスをインポートします。
local_model = SentenceTransformer('/kaggle/input/save-distilbert-base-nli-mean-tokens')  
# 保存したDistilBERTモデルをローカルパスから読み込みます。

modelky = KeyBERT(model=local_model)  
# KeyBERTのインスタンスを作成し、先ほど読み込んだモデルを使用します。これはキーワード抽出のための準備です。

In [ ]:
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')  # テストデータをCSVファイルから読み込みます。
# encoding='iso-8859-1'という引数は、必要に応じてコメントアウトされています。

# 新しい列をデータフレームに追加し、初期値を'-'に設定します。
test['prompt_kw'] = '-'  
test['res_a_kw'] = '-'  
test['res_b_kw'] = '-'  

# 'prompt'列からトップ5のキーワードを抽出します。
tkw0 = modelky.extract_keywords(test['prompt'], top_n=5)  
# 'response_a'列からトップ10のキーワードを抽出します。
tkw1 = modelky.extract_keywords(test['response_a'], top_n=10)  
# 'response_b'列からトップ10のキーワードを抽出します。
tkw2 = modelky.extract_keywords(test['response_b'], top_n=10)

In [ ]:
for i, w in enumerate(tkw0): 
    ws = []  # 空のリストを作成してキーワードを保存します。
    for wi in w:
        if '_' not in wi[0]:  # キーワードにアンダースコアが含まれていない場合
            ws += [wi[0]]  # キーワードをリストに追加します。
    test.loc[i, 'prompt_kw'] = ' '.join(ws)  # 抽出したキーワードをスペースで結合し、データフレームに保存します。
    
for i, w in enumerate(tkw1): 
    ws = []  # 空のリストを作成してキーワードを保存します。
    for wi in w:
        if '_' not in wi[0]:  # キーワードにアンダースコアが含まれていない場合
            ws += [wi[0]]  # キーワードをリストに追加します。
    test.loc[i, 'res_a_kw'] = ' '.join(ws)  # 抽出したキーワードをスペースで結合し、データフレームに保存します。

for i, w in enumerate(tkw2): 
    ws = []  # 空のリストを作成してキーワードを保存します。
    for wi in w:
        if '_' not in wi[0]:  # キーワードにアンダースコアが含まれていない場合
            ws += [wi[0]]  # キーワードをリストに追加します。
    test.loc[i, 'res_b_kw'] = ' '.join(ws)  # 抽出したキーワードをスペースで結合し、データフレームに保存します。

# 'res_a_kw'列のキーワードと'prompt_kw'列のキーワードを結合します。
test['res_a_kw'] = test['res_a_kw'] + ' // ' + test['prompt_kw']
# 'res_b_kw'列のキーワードと'prompt_kw'列のキーワードを結合します。
test['res_b_kw'] = test['res_b_kw'] + ' // ' + test['prompt_kw']
test = test.iloc[:, 4:]  # 最初の4列を除外して新しいデータフレームを作成します。
display(test)  # 最終的なデータフレームを表示します。

# test.to_csv('test_key.csv', index=False)  # （コメントアウトされた）データフレームをCSVファイルとして保存する行です。

In [ ]:
testA = test[['res_a_kw']]  # 'res_a_kw'列のみを抽出して新しいデータフレームtestAを作成します。
testA['label'] = 0  # 新しい列'label'を追加し、すべての値を0に設定します。
testA.columns = ['text', 'label']  # 列名を'text'と'label'に変更します。

testB = test[['res_b_kw']]  # 'res_b_kw'列のみを抽出して新しいデータフレームtestBを作成します。
testB['label'] = 0  # 新しい列'label'を追加し、すべての値を0に設定します。
testB.columns = ['text', 'label']  # 列名を'text'と'label'に変更します。

TEST = pd.concat([testA, testB], axis=0)  # testAとtestBを行方向（axis=0）で結合して新しいデータフレームTESTを作成します。

In [ ]:
max_sens = 8  # 最大文の数を8に設定します。これはデータの処理やモデル入力の際に使用される可能性があります。
p_test = TEST.reset_index(drop=True)  # TESTデータフレームのインデックスをリセットし、新しいインデックスでデータフレームp_testを作成します。

In [ ]:
class BERTDataSet(Dataset):
    # BERTに基づくデータセットクラスを定義します。このクラスはPyTorchのDatasetを継承しています。
    
    def __init__(self, sentences, targets):        
        # コンストラクタ：文とターゲットを初期化します。
        self.sentences = sentences  # 文を保存します。
        self.targets = targets  # 対応するターゲットを保存します。
        
    def __len__(self):        
        # データセットの長さ（文の数）を返します。
        return len(self.sentences)
    
    def __getitem__(self, idx):        
        # 指定されたインデックスの文とターゲットを取得します。
        sentence = self.sentences[idx]  
        
        # 文をBERTモデルの入力形式にエンコードします。
        bert_sens = tokenizer.encode_plus(
            sentence,
            add_special_tokens = True,  # 特殊トークンを追加します。
            max_length = max_sens,  # 最大長を設定します。
            pad_to_max_length = True,  # 最大長にパディングを行います。
            return_attention_mask = True  # 注意マスクを返します。
        )

        ids = torch.tensor(bert_sens['input_ids'], dtype=torch.long)  # 入力IDをテンソルに変換します。
        mask = torch.tensor(bert_sens['attention_mask'], dtype=torch.long)  # 注意マスクをテンソルに変換します。

        target = torch.tensor(self.targets[idx], dtype=torch.float)  # 対応するターゲットをテンソルに変換します。
        
        return {
            'ids': ids,  # 入力IDを戻します。
            'mask': mask,  # 注意マスクを戻します。
            'targets': target  # ターゲットを戻します。
        }

In [ ]:
test_dataset = BERTDataSet(p_test["text"], p_test["label"])  
# p_testの'text'と'label'を使用してBERTDataSetのインスタンスを作成します。このデータセットはテストデータを扱います。

test_batch = 32  # テストデータのバッチサイズを32に設定します。
test_dataloader = DataLoader(test_dataset, batch_size=test_batch, shuffle=False, num_workers=8, pin_memory=True)  
# DataLoaderを使用して、テストデータをバッチ処理します。
# - num_workers=8: データの読み込みを並列で処理するために8つのワーカーを使用します。
# - pin_memory=True: GPUへのデータ転送の効率を上げるためにメモリをピン留めします。

# 予測を行う関数
保存したモデルを使用します

In [ ]:
# モデルの初期化
tokenizer = transformers.RobertaTokenizer.from_pretrained("/kaggle/input/roberta-base")  
# 指定したパスからRoBERTaトークナイザーを読み込みます。

model = transformers.RobertaForSequenceClassification.from_pretrained("/kaggle/input/roberta-base", num_labels=1)  
# 指定したパスからRoBERTaモデルを読み込み、性質分類のためのモデルを作成します。
# num_labels=1: 出力ラベルの数を1に設定します。

# 指定したディレクトリ内の.pthファイルのパスをリストとして取得します。
pths = [os.path.join("/kaggle/input/lmsys-keywords-torch-roberta", s) for s in os.listdir("/kaggle/input/lmsys-keywords-torch-roberta") if ".pth" in s]
print(pths)  # 取得したパスを表示します。

In [ ]:
def predicting(
    test_dataloader,
    model,
    pths 
):
    # モデルを使用して予測を行う関数です。
    allpreds = []  # すべての予測結果を保存するリストです。    
    for pth in pths:  # 指定された各モデルのパスについてループします。  
        state = torch.load(pth, map_location=torch.device('cpu'))  # モデルの状態をロードします。
        model.load_state_dict(state["state_dict"])  # モデルにロードした状態を設定します。
        model.to(device)  # モデルをデバイス（GPUまたはCPU）に移動します。
        model.eval()  # モデルを評価モードに設定します。これにより、ドロップアウトやバッチ正規化が無効になります。      
        
        preds = []  # 各モデルの予測結果を保存するリストです。
        allvalloss = 0  # オプションで全体損失を蓄積するための変数ですが、現時点では使用されていません。

        with torch.no_grad():  # 勾配計算を無効にします（メモリの節約と計算速度の向上）。
            for a in test_dataloader:  # テストデータローダーからバッチを繰り返します。
                ids = a["ids"].to(device)  # バッチの入力IDをデバイスに移動します。
                mask = a["mask"].to(device)  # バッチの注意マスクをデバイスに移動します。
                output = model(ids, mask)  # モデルに入力を渡し、出力を取得します。
                output = output["logits"].squeeze(-1)  # 出力からロジットを取得し、次元を縮小します。
                preds.append(output.cpu().numpy())  # 予測をCPUに移動し、NumPy配列として追加します。

            preds = np.concatenate(preds)  # すべてのバッチの予測を結合します。           
            allpreds.append(preds)  # モデルによる予測をリストに追加します。

    return allpreds  # すべてのモデルからの予測結果を返します。

In [ ]:
tpreds = predicting(test_dataloader, model, pths)  
# テストデータローダー、モデル、およびモデルのパスを使って予測を行います。
# 予測結果はtpredsに保存されます。

# 予測結果

In [ ]:
test_pred = []  # 予測結果を保存するための空のリストを作成します。
for p in tpreds[0]:  # 最初のモデルの予測結果についてループします。
    test_pred += [p]  # それぞれの予測をリストに追加します。

In [ ]:
submit = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')  
# サンプル提出ファイルをCSVから読み込みます。

submit['winner_model_a'] = test_pred[0:len(test)]  # モデルAの勝者の予測を提出データフレームに追加します。
submit['winner_model_b'] = test_pred[len(test):]  # モデルBの勝者の予測を提出データフレームに追加します。

pa = submit['winner_model_a']  # モデルAの予測結果を変数に保存します。
pb = submit['winner_model_b']  # モデルBの予測結果を変数に保存します。

# モデルAとモデルBの予測の合計をクリップして、0から1の範囲に制限します（この値を勝者が引き分けた場合のための列に使用します）。
submit['winner_tie'] = np.clip((pa + pb), 0, 1)  
display(submit)  # 最終的な提出データフレームを表示します。

# 提出ファイルをCSV形式で保存します。
submit.to_csv('submission.csv', index=False)